In [1]:
import plotly.express as px
import plotly
plotly.io.templates.default = 'plotly_dark'
import pandas as pd
import sqlalchemy as sal

In [2]:
host="capstone.clihskgj8i7s.us-west-2.rds.amazonaws.com"
username="group3"
db="db1"
#pw=getpass.getpass("Enter database password")
pw=open("/home/daniel/Desktop/.awsdb","r").read().rstrip()

In [3]:
engine = sal.create_engine('postgresql://%s:%s@%s/%s' % (username, pw, host, db))

In [4]:
#download a snapshot of data
query="""
select l.sid, l.longitude, l.latitude, l.stype, l.fwy ,l.direc, l.abs_pm, t.total_flow ,
t.avg_speed
from traffic t, pemslocs l
where t.timestamp between '2021-01-22 15:00:00' and '2021-01-22 15:03:00'
and l.sid = t.station and l.stype = 'ML'
"""
snap=pd.read_sql(query, engine, index_col="sid")
snap.head()

,longitude,latitude,stype,fwy,direc,abs_pm,total_flow,avg_speed
sid,,,,,,,,
1108285,-117.134145,32.713318,ML,94,W,0.819,232,64.8
1108287,-117.127908,32.714211,ML,94,W,1.205,153,58.6
1108289,-117.119831,32.716724,ML,94,W,1.721,252,62.9
1108291,-117.117861,32.717134,ML,94,W,1.844,311,65.1
1108293,-117.113171,32.717909,ML,94,W,2.134,277,62.8


In [5]:
pd.read_sql("select * from pemslocs limit 1", engine)
#print(len(snap))

,sid,fwy,direc,district,county,city,state_pm,abs_pm,latitude,longitude,length,stype,lanes,name
0,308511,50,E,3,17,None,31.627,60.162,38.761062,-120.569835,5.0,ML,2,Sly Park Rd


In [6]:
snap["total_flow"]=snap["total_flow"].astype(float)

In [7]:
#create new column with fwy and direction as string
snap["fwdir"] = snap.apply(lambda x: str(x["fwy"]) + str(x["direc"]), axis=1)
snap["fwdir"].unique()

array(['94W', '125S', '78E', '163S', '8W', '8E', '125N', '94E', '805S',
       '78W', '15N', '5S', '15S', '5N', '805N', '163N', '54E', '54W',
       '56E', '56W', '52E', '52W', '67S', '67N', '905W', '905E', '76W',
       '76E', '11E', '11W'], dtype=object)

In [8]:
import pickle
with open("SD_fwys.pkl", "wb") as fid:
    pickle.dump(list(snap["fwdir"].unique()), fid)

In [74]:
#sel=snap.query('fwdir == "94W"').sort_values(by="abs_pm")
#sel=snap.query('fwy == "94"').sort_values(by="abs_pm")
sel=snap
sel=sel.sort_values(by=["fwdir","abs_pm"])
sel.tail()

,longitude,latitude,stype,fwy,direc,abs_pm,total_flow,avg_speed,fwdir,speed_cat
sid,,,,,,,,,,
1108315,-117.040554,32.744864,ML,94,W,7.130,365.0,63.7,94W,fast
1108317,-117.035279,32.745268,ML,94,W,7.454,343.0,64.4,94W,fast
1113173,-117.023229,32.746923,ML,94,W,8.204,278.0,64.4,94W,fast
1113126,-117.017105,32.748892,ML,94,W,8.611,139.0,67.0,94W,fast
1120121,-117.010444,32.753919,ML,94,W,9.140,97.0,63.2,94W,fast


In [88]:
def speed2cat(speed):
    if speed > 65:
        return 'fast'
    elif speed > 60:
        return 'moderate'
    elif speed > 50:
        return 'slow'
    else:
        return 'stopped'
        
sel["speed_cat"]=sel["avg_speed"].apply(speed2cat)

In [89]:
colorscales = px.colors.named_colorscales()
colorscales[0]

'aggrnyl'

In [90]:
"94W" != "94E"

True

In [91]:
#%%writefile road_group_split.py
def road_group_split(indf, splitcat="speed_cat", 
                     keep_cols=["latitude", "longitude", "abs_pm", "avg_speed","stype","fwdir"]):
    
    odict={}
    for col in keep_cols + [splitcat]:
        odict[col] = []
    #lats=[]
    #lons=[]
    #abs_pm=[]
    #cat=[]
    odict["linegroup"] = []
    
    p=0 #number holding line group
    i0=indf.index[0]
    
    for col in keep_cols + [splitcat]:
        odict[col].append(indf.loc[i0,col])
    
    #lats.append(indf.loc[i0, "latitude"])
    #lons.append(indf.loc[i0, "longitude"])
    #abs_pm.append(indf.loc[i0, "abs_pm"])
    #cat.append(indf.loc[i0, splitcat])
    #group.append(p)
    odict["linegroup"].append(p)
    
    for n in indf.index[1:]:
        if indf.loc[n,"fwdir"] != indf.loc[i0,"fwdir"]:
            p+=1
            
        if indf.loc[n,splitcat] != indf.loc[i0,splitcat] and indf.loc[n,"fwdir"] == indf.loc[i0,"fwdir"]:
            for col in keep_cols:
                odict[col].append(indf.loc[n,col])
            odict[splitcat].append(indf.loc[i0,splitcat])
            #end previous group with new point and speed cat from previous point
            #lats.append(indf.loc[n,"latitude"])
            #lons.append(indf.loc[n,"longitude"])
            #lons.append(indf.loc[n,"abs_pm"])
            #cat.append(indf.loc[i0,splitcat])
            #group.append(p)
            odict["linegroup"].append(p)
            p+=1
        #start new group
        for col in keep_cols + [splitcat]:
            odict[col].append(indf.loc[n,col])
        #lats.append(indf.loc[n,"latitude"])
        #lons.append(indf.loc[n,"longitude"])
        #cat.append(indf.loc[n,splitcat])
        #group.append(p)        
        odict["linegroup"].append(p)
        i0=n
    
    #odf=pd.DataFrame({"latitude": lats, "longitude": lons, splitcat: cat, "linegroup": group})
    odf=pd.DataFrame(odict)
    return odf

selg=road_group_split(sel)
selg.tail()

,latitude,longitude,abs_pm,avg_speed,stype,fwdir,speed_cat,linegroup
1015,32.746923,-117.023229,8.204,64.4,ML,94W,moderate,333
1016,32.748892,-117.017105,8.611,67.0,ML,94W,moderate,333
1017,32.748892,-117.017105,8.611,67.0,ML,94W,fast,334
1018,32.753919,-117.010444,9.140,63.2,ML,94W,fast,334
1019,32.753919,-117.010444,9.140,63.2,ML,94W,moderate,335


In [92]:
pd.set_option("display.max_rows", 50, "display.max_columns", 20)
selg

,latitude,longitude,abs_pm,avg_speed,stype,fwdir,speed_cat,linegroup
0,32.563648,-116.949472,0.304,48.4,ML,11E,stopped,0
1,32.564398,-116.949606,0.135,51.9,ML,11W,slow,1
2,32.564077,-116.942823,0.530,50.8,ML,11W,slow,1
3,32.564034,-116.938179,0.800,62.4,ML,11W,slow,1
4,32.564034,-116.938179,0.800,62.4,ML,11W,moderate,2
...,...,...,...,...,...,...,...,...
1015,32.746923,-117.023229,8.204,64.4,ML,94W,moderate,333
1016,32.748892,-117.017105,8.611,67.0,ML,94W,moderate,333
1017,32.748892,-117.017105,8.611,67.0,ML,94W,fast,334
1018,32.753919,-117.010444,9.140,63.2,ML,94W,fast,334


In [93]:
cdm={"fast": "green",
     "moderate": "orange",
     "slow": "red",
     "very slow": "brown"}

In [94]:
fig = px.line_mapbox(selg, lat="latitude", lon="longitude", color="speed_cat",
                        #mapbox_style="carto-darkmatter",
                        mapbox_style="stamen-terrain",
                        line_group="linegroup",
                        #color_continuous_scale="aggrnyl" ,
                        hover_data=["stype", "abs_pm", "linegroup", "fwdir"],
                        center = {"lat": 32.77, "lon": -117.16},
                        color_discrete_map= cdm,
                        zoom=12,
                        height=800, width=800)

for dat in fig.data:
    dat["line"]["width"] = 3
fig
#px.scatter(sel, x="longitude", y="latitude", color="total_flow").update_traces(mode='lines+markers')

In [95]:
fig.write_image("traffic_layer_wo_usdot.jpg")

In [67]:
#myshp = open("*.shp", "rb")
#px.scatter_mapbox?
fig.data[1]

Scattermapbox({
    'customdata': array([['ML', 0.456, 2, '94W'],
                         ['ML', 0.819, 2, '94W'],
                         ['ML', 1.083, 2, '94W'],
                         ['ML', 1.205, 2, '94W']], dtype=object),
    'hovertemplate': ('speed_cat=fast<br>linegroup=%{' ... '{customdata[3]}<extra></extra>'),
    'lat': array([32.713274, 32.713318, 32.713724, 32.714211]),
    'legendgroup': 'fast',
    'line': {'color': '#636efa'},
    'lon': array([-117.140097, -117.134145, -117.129847, -117.127908]),
    'mode': 'lines',
    'name': 'fast',
    'showlegend': False,
    'subplot': 'mapbox'
})

In [70]:
fig.data[0]["line"]["width"] = 10